In [1]:
import os
%pwd

'/Users/slackroo/Data_science/GitHub/kidney_disease_classification_deeplearning/research'

In [2]:
os.chdir("../")
%pwd

'/Users/slackroo/Data_science/GitHub/kidney_disease_classification_deeplearning'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class NovelBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    param_image_size: list
    param_learning_rate: float
    param_include_top: bool
    param_weights: str
    param_classes: int

In [4]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_novel_base_model_config(self)-> NovelBaseModelConfig:
        config = self.config.novel_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = NovelBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            param_image_size=self.params.IMAGE_SIZE,
            param_learning_rate=self.params.LEARNING_RATE,
            param_include_top=self.params.INCLUDE_TOP,
            param_weights=self.params.WEIGHTS,
            param_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [5]:
# workflow configuration manager
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
class NovelBaseModel:
    def __init__(self, config: NovelBaseModelConfig):
        self.config = config

    @staticmethod
    def save_model(path:Path,model: tf.keras.Model):
        model.save(path)

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.param_image_size,
            weights=self.config.param_weights,
            include_top=self.config.param_include_top
        )
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes=self.config.param_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.param_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

2023-10-08 17:50:06.358866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/slackroo/Data_science/GitHub/kidney_disease_classification_deeplearning/venv/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
# for workflow update pipeline

try:
    config = ConfigurationManager()
    novel_base_model_config = config.get_novel_base_model_config()
    novel_base_model = NovelBaseModel(config=novel_base_model_config)
    novel_base_model.get_base_model()
    novel_base_model.update_base_model()
except Exception as e:
    raise e

[2023-10-08 17:50:10,735: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-08 17:50:10,738: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-08 17:50:10,739: INFO: common: created directory at: artifacts]
[2023-10-08 17:50:10,740: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-10-08 17:50:11,208: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64